In [74]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import time
import pandas as pd
import re
import os

무신사 브랜드 내용가져오기

In [58]:
options = webdriver.ChromeOptions()

options.add_argument('--disable-dev-shm-usage') 
options.add_argument("--no-sandbox") 
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)


data = []
for page_num in tqdm(range(1,41), desc='pages'):
    url = f'https://www.musinsa.com/brands?categoryCode=001&type=&sortCode=BRAND_RANK&page={page_num}&size=100'
    driver.get(url)
    
    # 한 페이지의 모든 브랜드 정보를 로드할 때까지 기다림
    wait = WebDriverWait(driver, 6)
    wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="text_list"]/ul/li')))

    for i in tqdm(range(1, 101), desc='Brands', leave=False):  # tqdm으로 브랜드 진행 상황 표시
        try:
            Brand_KR = driver.find_element(By.XPATH, f'//*[@id="text_list"]/ul/li[{i}]/dl/dt/a').text
            Brand_ENG_Element = driver.find_element(By.XPATH, f'//*[@id="text_list"]/ul/li[{i}]/dl/dd/a')
            Brand_ENG = Brand_ENG_Element.text.split('(')[0].strip() 
            Count = Brand_ENG_Element.find_element(By.TAG_NAME, 'span').text 
            Count = Count.replace('(', '').replace(')', '').replace(',', '')

            data.append([Brand_KR, Brand_ENG, Count])  # 데이터 리스트에 추가
        except NoSuchElementException:
            continue
driver.quit()

# 데이터프레임 생성 및 출력
df = pd.DataFrame(data, columns=['Brand_KR', 'Brand_ENG', '개수'])

df.to_csv('BrandName.csv', index=False)



pages: 100%|██████████| 40/40 [03:37<00:00,  5.44s/it]


브랜드 이미지, 정보,  since 가져오기

In [81]:
df = pd.read_csv('BrandName.csv')
brand = df['Brand_ENG'].tolist()
print(brand)
brand_name = []
for i in brand:
    i = i.replace(' ','')
    brand_name.append(i)
print(brand_name)

['MUSINSASTANDARD', 'ASICS', 'ADIDAS', 'LMOOD', 'LAFUDGESTORE', 'TOFFEE', 'TRILLION', 'DRAW FIT', 'CODEGRAPHY', 'YALE', 'GLW', 'SATUR', 'DIMITRI BLACK', 'ESPIONAGE', 'LAFUDGE FOR WOMAN', 'DOFFJASON', 'THE NORTH FACE', 'NEW BALANCE', 'FRIZMWORKS', 'OUTSTANDING', 'GAKKAI UNIONS', 'FILLUMINATE', 'TRAVEL', 'XERO', 'MATIN KIM', 'THISISNEVERTHAT', 'SPAO', 'UMBRO', 'WHATITISNT', 'LEE', 'POLO RALPH LAUREN', 'COVERNAT', 'SUARE', 'SOVERMENT', 'UNIFORM BRIDGE', 'NATIONALGEOGRAPHIC', 'PARTIMENTO', '1993STUDIO', 'PUMA', 'DISCOVERY EXPEDITION', 'BRANDED', 'COOR', 'NOT4NERD', 'LIKE THE MOST', 'CPGN STUDIO', 'AVANDRESS', 'INTEMPOMOOD', 'URBANDTYPE', 'KIIMUIR', 'STANDARDERROR', 'JEMUT', 'EZKATON', 'UNION BLUE', 'INSILENCE', 'SPYDER', 'NOMANUAL', 'SCULPTOR', 'CALVIN KLEIN JEANS', 'GLOWNY', 'MLB', 'PLACE STUDIO', 'SIGNATURE', 'HOKA', 'PHYSICAL EDUCATION DEPARTMENT', 'FABREGAT', 'FP142', 'DICKIES', 'MARDI MERCREDI', 'GIORDANO', 'SLOW ACID', 'BROWNBREATH', 'MODNINE', 'IL CORSO', 'TAKEASY', '8SECONDS', 'NON

In [82]:
def save_progress(data, filename='Brand_Info.csv'):
    df = pd.DataFrame(data, columns=['Brand', 'Logo', 'Since', 'Country', 'Info'])
    if not os.path.exists(filename):  
        df.to_csv(filename, index=False)
    else:  
        df.to_csv(filename, mode='a', header=False, index=False)

def load_completed_brands(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename)['Brand'].tolist()
    except FileNotFoundError:
        return []

def save_completed_brands(brands, filename='completed_brands.csv'):
    pd.DataFrame(brands, columns=['Brand']).to_csv(filename, index=False)

options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage') 
options.add_argument("--no-sandbox") 
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)
time.sleep(1)
completed_brands = load_completed_brands()


for brand in tqdm(brand_name, desc='Brands'):
    if brand in completed_brands:
        continue 
    
    url = f'https://www.musinsa.com/brands/{brand}'
    driver.get(url)

    try:
        logo_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/p[1]/img')
        since_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/p[2]')
        country_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/div/span/img')
        info_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[2]/div[3]/div[1]/span[1]')
       
        logo = logo_element.get_attribute('src') 
        since = since_element.text
        country = country_element.get_attribute('alt') 
        info = info_element.text
        
        # 각 브랜드 처리 후 바로 CSV 파일에 저장
        save_progress([[brand, logo, since, country, info]])
        
        # 처리 완료된 브랜드 목록 업데이트
        completed_brands.append(brand)
        save_completed_brands(completed_brands)

    except NoSuchElementException:
        continue

driver.quit()

Brands: 100%|██████████| 3993/3993 [1:26:48<00:00,  1.30s/it]


상품들 가져오기

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm
import time
import pandas as pd
import re
import os

In [ ]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver


def load_progress(filename='completed_brands.csv'):
    try:
        df = pd.read_csv(filename)
        if df.empty:
            return None, 1
        last_row = df.iloc[-1]
        brand = last_row['Brand']
        last_completed_page = last_row['Page_Number']
        return brand, last_completed_page
    except FileNotFoundError:
        return None, 1


def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)


def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False


def scrape_product_info(driver, brand ,item):
    item_price = None
    item_details = {
        "brand": brand,
        "item_category": None,
        "item_img": None,
        "item_name": None,
        "item_num": None,
        "item_brand": None,
        "item_price": None
    }

    time.sleep(0.5)

    for div_num in range(4,1,-1):
        try:
            item_price_element =driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[{div_num}]/div[2]/p[3]')
            item_price = item_price_element.text
            break
        except NoSuchElementException:
            pass

    time.sleep(0.5)

    for div_num in range(1,5):
        try:
            item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[{div_num}]/div[2]/p[2]/a')
            driver.execute_script("arguments[0].click();", item_element)
            break
        except NoSuchElementException:
            pass
   
    
    time.sleep(0.5)
    
    for num in range(1,5):
        try:
            item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
            
            item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
            
            item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
            
            item_details["item_num"] = driver.find_element(By.XPATH, f'//*[@id="root"]/div[1]/div[2]/div[2]/div[{num}]/ul/li[1]/div[2]/span[2]').text
            break
        except NoSuchElementException:
            pass

    for div_num in range(1,5):
        try:
            item_details["item_brand"] = driver.find_element(By.XPATH, f'//*[@id="root"]/div[1]/div[2]/div[2]/div[{div_num}]/ul/li[1]/div[2]/a').text
            item_details["item_price"] = item_price
            break
        except NoSuchElementException:
            pass
        
    driver.back()
    return item_details


def save_data(data, brand, page_num, brand_page_info='Items_Info.csv'):
    if data:
        new_data = pd.DataFrame(data)
        brand_page_info = f"./Datasets/{brand}_page_{page_num}.csv"
        if not os.path.exists("./Datasets"):
            os.makedirs("./Datasets")
        if os.path.exists(brand_page_info):
            
            existing_data = pd.read_csv(brand_page_info)
            if not existing_data.empty:
                
                return
        
        new_data.to_csv(brand_page_info, mode='a', header=not os.path.exists(brand_page_info), index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, start_page=1, end_page=100):
    completed_brands = []
    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
           
            item_details = scrape_product_info(driver, brand, item)
            if item_details:
                data.append(item_details)
            else:
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, brand, page_num)
        data.clear()
        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)  

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand_Name.csv')
    brand_names = df['Brand_ENG'].tolist()
    last_completed_brand, start_page = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        if brand == last_completed_brand:
            start_page += 1  
        else:
            start_page = 1  
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, start_page=start_page)
        start_page = 1  

    driver.quit()


if __name__ == "__main__":
    main()

In [2]:
import os

In [4]:
csvlist = os.listdir('../Datasets/')
csvlist[:5]

['1993STUDIO_page_1.csv',
 '1993STUDIO_page_2.csv',
 '1993STUDIO_page_3.csv',
 'BRANDED_page_1.csv',
 'BRANDED_page_2.csv']

In [5]:
import pandas as pd

In [12]:
import pandas as pd

file = []

# item_info.csv 파일 읽기
data = pd.read_csv('item_info.csv')

for filename in csvlist:
    # 각 CSV 파일 읽기
    a = pd.read_csv(f'../Datasets/{filename}')
    
    # 데이터 프레임 합치기
    combine = pd.concat([data, a], ignore_index=True)
    
    # 리스트에 추가
    file.append(combine)

# 모든 데이터프레임을 합친 후 CSV 파일로 저장
result = pd.concat(file, ignore_index=True)

result.to_csv('file.csv', index=False)


In [13]:
data = pd.read_csv('items.csv')
data[:5]

,brand,item_category,item_img,item_name,item_num,item_brand,item_price
0,1993STUDIO,스포츠/용품,https://image.msscdn.net/images/goods_img/2024...,[LG트윈스] 쿨코튼 V3 챔피언 티셔츠_화이트,93242TTS100102,1993STUDIO,"39,000원 31,200원"
1,1993STUDIO,스포츠/용품,https://image.msscdn.net/images/goods_img/2024...,[LG트윈스] 쿨코튼 V3 챔피언 티셔츠_블랙,93242TTS100102,1993STUDIO,"39,000원 31,200원"
2,1993STUDIO,상의,https://image.msscdn.net/images/goods_img/2024...,93 파이핑 풋볼 저지_블루,93241ETS110116,1993STUDIO,"69,000원 55,200원"
3,1993STUDIO,상의,https://image.msscdn.net/images/goods_img/2024...,93 파이핑 풋볼 저지_화이트,93241ETS110102,1993STUDIO,"69,000원 55,200원"
4,1993STUDIO,상의,https://image.msscdn.net/images/goods_img/2024...,93 파이핑 풋볼 저지_핑크,93241ETS110149,1993STUDIO,"69,000원 55,200원"


In [18]:
# 각 열의 결측치 수 확인
print(data.isnull().sum())


brand                0
item_category      143
item_img           143
item_name          143
item_num           143
item_brand       13058
item_price         217
dtype: int64


In [22]:
data.dropna(subset=['item_category', 'item_img', 'item_name', 'item_num', 'item_brand', 'item_price'], inplace=True)


In [23]:
data.isnull().sum()


brand            0
item_category    0
item_img         0
item_name        0
item_num         0
item_brand       0
item_price       0
dtype: int64

In [ ]:

data.to_csv('f.csv')